Pypeline de Dados: Combinando Python com Orientação a Objetivo

In [1]:
import csv
import json

# --- CONFIGURAÇÃO DOS CAMINHOS ---
# Centralizamos os caminhos aqui para facilitar mudanças futuras
path_csv = '../data_raw/dados_empresaB.csv'
path_json = '../data_raw/dados_empresaA.json'
path_dados_combinados = '../data_processed/dados_combinados.csv'

# --- ETAPA 1: Leitura do CSV (Extract) ---
print("Iniciando leitura do CSV...")
dados_csv = []
with open(path_csv, 'r') as file:
    spamreader = csv.DictReader(file, delimiter=',')
    # List Comprehension para carregar os dados
    dados_csv = [row for row in spamreader]

# --- ETAPA 2: Leitura do JSON (Extract) ---
print("Iniciando leitura do JSON...")
with open(path_json, 'r') as file:
    dados_json = json.load(file)

# --- ETAPA 3: Transformação (Transform) ---
print("Iniciando transformação...")

key_mapping = {
    'Nome do Item': 'Nome do Produto',
    'Classificação do Produto': 'Categoria do Produto',
    'Valor em Reais (R$)': 'Preço do Produto (R$)',
    'Quantidade em Estoque': 'Quantidade em Estoque',
    'Nome da Loja': 'Filial',
    'Data da Venda': 'Data da Venda'
}

# Aqui está a correção crucial no .get()
new_dados_csv = [
    {
        # Tenta encontrar a tradução. Se não achar, usa a chave original (old_key).
        # Isso previne que colunas novas virem "None" ou sumam.
        key_mapping.get(old_key, old_key): value 
        for old_key, value in old_dict.items()
    }
    for old_dict in dados_csv
]

# --- ETAPA 4: Combinação e Preparação para Carga (Load) ---
combined_list = new_dados_csv + dados_json
print(f"Total de registros combinados: {len(combined_list)}")

# DESCCOBRINDO TODAS AS COLUNAS (Solução do Erro do DictWriter)
# Em vez de pegar só o primeiro item, vamos varrer todos para garantir
# que pegamos todas as colunas possíveis, mesmo as que aparecem raramente.
nome_colunas = set() # Usamos set (conjunto) para não ter nomes repetidos
for row in combined_list:
    nome_colunas.update(row.keys())

# Convertemos de volta para lista para o DictWriter usar
nome_colunas = list(nome_colunas)
print(f"Colunas finais identificadas: {nome_colunas}")

# --- ETAPA 5: Salvando o Arquivo (Load) ---
print("Salvando arquivo final...")
with open(path_dados_combinados, 'w', newline='', encoding='utf-8') as file:
    # O 'extrasaction' é uma segurança extra. Se 'ignore', ele ignora colunas extras.
    # Mas como mapeamos todas as colunas acima, não deve ser necessário, 
    # mas é bom conhecer. Aqui vamos usar o padrão (raise error) pois já tratamos.
    writer = csv.DictWriter(file, fieldnames=nome_colunas)
    
    writer.writeheader()
    writer.writerows(combined_list) # writerows (plural) é mais rápido que o loop for

print(f"Sucesso! Arquivo salvo em: {path_dados_combinados}")

Iniciando leitura do CSV...
Iniciando leitura do JSON...
Iniciando transformação...
Total de registros combinados: 4446
Colunas finais identificadas: ['Filial', 'Preço do Produto (R$)', 'Nome do Produto', 'Data da Venda', 'Categoria do Produto', 'Quantidade em Estoque']
Salvando arquivo final...
Sucesso! Arquivo salvo em: ../data_processed/dados_combinados.csv


Vamos clarear isso juntos! É perfeitamente normal sentir que essa sintaxe é um pouco "densa" no início, especialmente quando misturamos a biblioteca `csv` com as *comprehensions* (aquelas construções resumidas em uma linha).

Para facilitar, vamos dividir os trechos que você mandou em três momentos lógicos do processamento de dados: **Entrada (Leitura)**, **Processamento (Transformação)** e **Saída (Escrita)**.

### 1. A Entrada: Lendo os Dados (`DictReader`)

O primeiro passo é tirar os dados do arquivo de texto e trazê-los para o mundo do Python.

```python
with open(path_csv, 'r') as file:
    spamreader = csv.DictReader(file, delimiter=',')
    # List Comprehension para carregar os dados
    dados_csv = [row for row in spamreader]

```

* **`csv.DictReader`**: Imagine que ele é um tradutor. Ele lê uma linha de texto do CSV (ex: `Bola, 10, Azul`) e a converte automaticamente em um **dicionário** usando o cabeçalho do arquivo como chaves (ex: `{'Produto': 'Bola', 'Preço': '10', 'Cor': 'Azul'}`).
* **`dados_csv = [row for row in spamreader]`**: O `DictReader` é "preguiçoso"; ele lê uma linha por vez apenas quando pedido. Essa linha de código (uma *list comprehension*) diz: "Percorra o leitor (`spamreader`) até o fim, pegue cada linha (`row`) e guarde todas elas numa lista chamada `dados_csv`". Agora você tem todos os dados na memória RAM.

### 2. O Processamento: A Transformação Aninhada

Este é o trecho mais complexo, pois tem um loop dentro de outro loop.

```python
# (Trecho simplificado para foco)
new_dados = [
    {key_mapping.get(old_key, old_key): value for old_key, value in old_dict.items()}
    for old_dict in dados_csv
]

```

Vamos ler de **fora para dentro**:

1. **O Loop Externo (`for old_dict in dados_csv`)**: Estamos pegando cada linha (dicionário) da nossa lista original. É como pegar uma caixa de uma pilha.
2. **O Loop Interno (`for old_key, value in old_dict.items()`)**: Agora, *dentro* daquela linha específica, olhamos para cada par de Chave/Valor.
3. **A Construção (`{ ... }`)**: Estamos criando um *novo* dicionário. A mágica acontece aqui: pegamos o valor antigo (`value`), mas trocamos a chave antiga (`old_key`) por uma nova usando o `key_mapping`.

É basicamente uma "fábrica" que pega uma lista de dicionários antigos e cospe uma lista de dicionários novos com as chaves traduzidas.

### 3. A Saída: Escrevendo o Resultado (`DictWriter`)

Agora que temos a lista combinada e limpa (`combined_list`), precisamos salvar em arquivo.

```python
writer = csv.DictWriter(file, fieldnames=nome_colunas)
writer.writeheader()
writer.writerows(combined_list)

```

* **`writer = csv.DictWriter(...)`**: Aqui estamos configurando a máquina de escrever. O argumento mais importante é `fieldnames=nome_colunas`. O Python *precisa* saber a ordem exata das colunas para escrever o CSV corretamente. Se você tiver um dado extra no dicionário que não está em `fieldnames`, ele vai dar erro (ou ignorar, dependendo da configuração).
* **`writer.writeheader()`**: Escreve a primeira linha do arquivo (os títulos das colunas: "Nome", "Preço", etc.) baseada nos `fieldnames` que você definiu acima.
* **`writer.writerows(combined_list)`**: O comando final. Ele pega sua lista de dicionários e escreve todos de uma vez no arquivo, garantindo que cada valor vá para a coluna certa, separado por vírgulas.

---

Para garantir que a lógica do **DictWriter** ficou clara:

Se na sua lista `combined_list` você tiver um dicionário com a chave "Peso", mas você **não** incluiu "Peso" na lista `nome_colunas` (fieldnames), o que você acha que acontece quando o `writer.writerows` tenta salvar esse dado?



Essa "mágica" acontece por causa de como o método .get() foi construído no Python. Ele aceita dois "ingredientes" (argumentos) dentro dos parênteses, separados por vírgula.

Vamos "traduzir" o que o Python lê quando vê key_mapping.get(old_key, old_key):

    O primeiro old_key: É a pergunta. "Python, por favor, procure essa chave no dicionário key_mapping."

    O segundo old_key: É o plano de emergência (o valor padrão). "Python, se você não encontrar essa chave, me devolva este valor aqui no lugar de dar erro ou None."

Visualizando o processo

Imagine que o key_mapping é um tradutor.
Situação	Código	O que o Python pensa	Resultado
Chave existe	.get('Nome do Item', ...)	"Achei! A tradução é 'Nome do Produto'."	'Nome do Produto'
Chave nova	.get('País', 'País')	"Não achei 'País' no mapa. Vou devolver o valor padrão."	'País'

Se você tivesse escrito apenas key_mapping.get(old_key), o "plano de emergência" padrão do Python seria devolver None (vazio), o que causaria aqueles problemas que discutimos antes.

In [9]:
frutas = {'apple': 'maçã', 'banana': 'banana'}

# Tentando buscar 'orange'
resultado = frutas.get('orange', 'desconhecido')

print(f"O resultado é: {resultado}")
print(f"O dicionário frutas é: {frutas}")

O resultado é: desconhecido
O dicionário frutas é: {'apple': 'maçã', 'banana': 'banana'}


É por isso que essa técnica é tão segura para o seu key_mapping.

Quando fazemos: key_mapping.get(old_key, old_key)

Estamos dizendo: "Se você não tiver uma tradução para esta coluna, apenas me devolva o nome original dela agora. Não precisa mudar o mapa de traduções."
